In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from deap import base, creator, tools, algorithms
from discretize import TensorMesh
from SimPEG import maps
from SimPEG.electromagnetics import time_domain as tdem
from joblib import Parallel, delayed
import multiprocessing

# ----------------------------------------------
# 1. Cargar datos observados
# ----------------------------------------------
times = np.logspace(-5, -2, 31)
dobs = np.loadtxt("Datos_5.txt")[:, 1]
uncertainties = 0.03 * np.abs(dobs)

# Geometría del experimento
source_location = np.array([0.0, 0.0, 1.0])
receiver_location = np.array([0.0, 0.0, 1.0])
receiver_list = [tdem.receivers.PointMagneticFluxDensity(receiver_location, times, orientation="z")]
source_list = [tdem.sources.CircularLoop(
    receiver_list=receiver_list,
    location=source_location,
    waveform=tdem.sources.StepOffWaveform(),
    current=1.0,
    radius=10.0)]
survey = tdem.Survey(source_list)

# ----------------------------------------------
# 2. Función para graficar modelo 1D
# ----------------------------------------------
def plot_1d_layer_model(thicknesses, conductivities, ax=None, color='b', linestyle='-', label=None):
    if ax is None:
        ax = plt.gca()
    z = np.r_[0, np.cumsum(thicknesses)]
    sigma = np.r_[conductivities[0], conductivities]
    for i in range(len(thicknesses)):
        ax.hlines(z[i], sigma[i], sigma[i+1], color=color, linestyle=linestyle,
                  label=label if i == 0 else None)
        ax.vlines(sigma[i+1], z[i], z[i+1], color=color, linestyle=linestyle)
    ax.hlines(z[-1], sigma[-1], sigma[-1], color=color, linestyle=linestyle)

def plot_results(best_model, thicknesses, mapping, true_model=None):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    plot_1d_layer_model(thicknesses, mapping * best_model, ax=ax1, color='r', label='Modelo Invertido')

    if true_model is not None:
        true_thicknesses = np.array([20.0, 30.0, 40.0, 50.0])
        plot_1d_layer_model(true_thicknesses, true_model, ax=ax1, color='k', linestyle='--', label='Modelo Verdadero')

    ax1.set_xscale('log')
    ax1.set_xlabel('Conductividad (S/m)')
    ax1.set_ylabel('Profundidad (m)')
    ax1.invert_yaxis()
    ax1.grid(True, which='both')
    ax1.legend()
    ax1.set_title('Comparación de Modelos')

    simulation = tdem.Simulation1DLayered(
        survey=survey,
        thicknesses=thicknesses,
        sigmaMap=mapping
    )
    dpred = simulation.dpred(best_model)

    ax2.loglog(times, np.abs(dobs), 'ko', label='Datos Observados')
    ax2.loglog(times, np.abs(dpred), 'r-', label='Respuesta del Modelo')
    ax2.set_xlabel('Tiempo (s)')
    ax2.set_ylabel('dB/dt (T/s)')
    ax2.grid(True, which='both')
    ax2.legend()
    ax2.set_title('Ajuste de Datos TEM')

    plt.tight_layout()
    plt.show()

def plot_convergence(logbook):
    plt.figure(figsize=(8, 5))
    plt.plot(logbook.select("min"), label="Min Misfit")
    plt.plot(logbook.select("avg"), label="Avg Misfit")
    plt.xlabel("Generación")
    plt.ylabel("Costo")
    plt.yscale("log")
    plt.grid(True, which="both")
    plt.title("Convergencia del AG")
    plt.legend()
    plt.tight_layout()
    plt.show()

def penalized_cost(misfit, n_params, n_data):
    return misfit + 2 * n_params  # AIC

# ----------------------------------------------
# 3. Inversión con AG optimizado
# ----------------------------------------------
def run_inversion_n_layers(n_layers):
    inv_thicknesses = np.full(n_layers - 1, 20.0)
    mesh = TensorMesh([(np.r_[inv_thicknesses, inv_thicknesses[-1]])], "0")
    model_mapping = maps.ExpMap()
    starting_model = np.log(1 / 100 * np.ones(mesh.nC))

    simulation = tdem.Simulation1DLayered(
        survey=survey,
        thicknesses=inv_thicknesses,
        sigmaMap=model_mapping
    )

    if not hasattr(creator, "FitnessMin"):
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    if not hasattr(creator, "Individual"):
        creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    log_min, log_max = np.log(1/10), np.log(1/0.01)
    toolbox.register("attr_float", np.random.uniform, log_min, log_max)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=mesh.nC)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    toolbox.register("mate", tools.cxBlend, alpha=0.3)
    toolbox.register("mutate", tools.mutPolynomialBounded,
                     low=log_min, up=log_max, eta=10.0, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    def evaluate(individual):
        model = np.array(individual)
        try:
            dpred = simulation.dpred(model)
            misfit = np.sum(((dpred - dobs) / uncertainties)**2)
            roughness = np.sum(np.diff(model)**2)
            return misfit + 0.5 * roughness,
        except:
            return 1e12,

    toolbox.register("evaluate", evaluate)

    population = toolbox.population(n=150)
    stats = tools.Statistics(lambda ind: ind.fitness.values[0])
    stats.register("avg", np.mean)
    stats.register("min", np.min)

    result, logbook = algorithms.eaMuPlusLambda(
        population, toolbox,
        mu=100, lambda_=100,
        cxpb=0.2, mutpb=0.4,
        ngen=80,
        stats=stats,
        verbose=False
    )

    best_ind = tools.selBest(result, 1)[0]
    best_model = np.array(best_ind)
    misfit_final = toolbox.evaluate(best_ind)[0]

    return best_model, logbook, misfit_final, inv_thicknesses, model_mapping

# ----------------------------------------------
# 4. Paralelismo para distintas capas
# ----------------------------------------------
def run_and_process_layers(n_layers):
    print(f" Iniciando inversión para {n_layers} capas...")
    best_model, logbook, misfit, thicknesses, mapping = run_inversion_n_layers(n_layers)
    n_params = len(best_model)
    cost = penalized_cost(misfit, n_params, len(dobs))
    print(f"Finalizada inversión para {n_layers} capas. Misfit: {misfit:.4e}, AIC: {cost:.4e}")
    return {
        "n_layers": n_layers,
        "model": best_model,
        "logbook": logbook,
        "misfit": misfit,
        "aic": cost,
        "thicknesses": thicknesses,
        "mapping": mapping
    }

# ----------------------------------------------
# 5. Ejecución Principal
# ----------------------------------------------
if __name__ == "__main__":
    true_model = np.r_[0.09, 0.7, 0.05, 0.2, 0.01]  # Modelo real

    layer_range = range(2, 9)
    print(f"Usando {multiprocessing.cpu_count()} núcleos...")

    results = Parallel(n_jobs=-1)(delayed(run_and_process_layers)(n) for n in layer_range)

    best_result = min(results, key=lambda r: r["aic"])

    print("\n✅ ¡Modelo óptimo encontrado por AIC!")
    print("Capas:", best_result["n_layers"])
    print("Conductividades (S/m):", best_result["mapping"] * best_result["model"])

    plot_results(best_result["model"], best_result["thicknesses"],
                 best_result["mapping"], true_model=true_model)
    plot_convergence(best_result["logbook"])

    # Plot Misfit y AIC vs número de capas
    capas = [r["n_layers"] for r in results]
    errores = [r["misfit"] for r in results]
    aics = [r["aic"] for r in results]
    best_n_layers = best_result["n_layers"]
    best_aic = best_result["aic"]
    best_misfit = best_result["misfit"]

    plt.figure(figsize=(10, 5))
    plt.plot(capas, errores, 'o-k', label='Misfit regularizado')
    plt.plot(capas, aics, 'o--r', label='AIC')
    plt.plot(best_n_layers, best_misfit, 'k*', markersize=15, label='Mejor Misfit')
    plt.plot(best_n_layers, best_aic, 'r*', markersize=15, label='Mejor AIC')
    plt.xlabel("Número de capas")
    plt.ylabel("Costo")
    plt.grid(True)
    plt.title("Misfit y AIC vs Número de Capas")
    plt.legend()
    plt.tight_layout()
    plt.show()


Usando 12 núcleos...
